# LeRobot ALOHA Environment Demo

This notebook demonstrates the gym-aloha environment from LeRobot. ALOHA (A Low-cost Open-source Hardware System for Bimanual Teleoperation) is a bimanual manipulation environment with tasks like insertion and cube transfer.

## Import Required Libraries

In [ ]:
!pip install matplotlib pandas seaborn scikit-learn
!pip install IPython
# Install gym-aloha environment
!pip install 'lerobot[aloha]'

**Note:** After running the installation cell above, you may need to restart the kernel if you get compatibility errors.

In [ ]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time

In [ ]:
# Suppress deprecation warnings
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

In [ ]:
# Import gym_aloha to register the environment
import gym_aloha

## Available ALOHA Tasks

The gym-aloha environment provides several tasks:
- **AlohaInsertion-v0**: Insert a peg into a socket
- **AlohaTransferCube-v0**: Transfer a cube from one location to another

These tasks involve bimanual manipulation with a simulated ALOHA robot.

## Create the ALOHA Environment

In [ ]:
# Create the environment - Try AlohaInsertion-v0 first
task = "AlohaInsertion-v0"  # or "AlohaTransferCube-v0"
env = gym.make(f'gym_aloha/{task}', render_mode='rgb_array')

print(f"Task: {task}")
print(f"Observation space: {env.observation_space}")
print(f"Action space: {env.action_space}")

## Reset Environment and Visualize Initial State

In [ ]:
# Reset the environment
observation, info = env.reset(seed=42)

# Render and display the initial state
frame = env.render()

plt.figure(figsize=(10, 8))
plt.imshow(frame)
plt.title(f"Initial State of {task}")
plt.axis('off')
plt.show()

# Display observation information
if isinstance(observation, dict):
    print("\nObservation keys:")
    for key, value in observation.items():
        if isinstance(value, np.ndarray):
            print(f"  {key}: shape {value.shape}")
        else:
            print(f"  {key}: {type(value)}")
else:
    print(f"\nObservation shape: {observation.shape}")

## Run Random Actions

In [ ]:
# Run a few steps with random actions
num_steps = 100
total_reward = 0

observation, info = env.reset(seed=42)

for step in range(num_steps):
    # Sample a random action
    action = env.action_space.sample()
    
    # Step the environment
    observation, reward, terminated, truncated, info = env.step(action)
    total_reward += reward
    
    # Break if episode ends
    if terminated or truncated:
        print(f"Episode ended at step {step + 1}")
        break

print(f"\nTotal steps: {step + 1}")
print(f"Total reward: {total_reward:.4f}")
print(f"Terminated: {terminated}, Truncated: {truncated}")

## Visualize Episode with Random Actions

In [ ]:
# Collect frames for visualization
frames = []
rewards = []
num_steps = 100

observation, info = env.reset(seed=123)
frames.append(env.render())

for step in range(num_steps):
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)
    
    frames.append(env.render())
    rewards.append(reward)
    
    if terminated or truncated:
        break

print(f"Collected {len(frames)} frames")
print(f"Total reward: {sum(rewards):.4f}")

## Display Selected Frames

In [ ]:
# Display 6 frames evenly spaced throughout the episode
num_display = 6
indices = np.linspace(0, len(frames) - 1, num_display, dtype=int)

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, idx in enumerate(indices):
    axes[i].imshow(frames[idx])
    axes[i].set_title(f"Step {idx}")
    axes[i].axis('off')

plt.tight_layout()
plt.show()

## Plot Reward Over Time

In [ ]:
# Plot the rewards
plt.figure(figsize=(12, 4))
plt.plot(rewards, label='Reward per step')
plt.plot(np.cumsum(rewards), label='Cumulative reward', linestyle='--')
plt.xlabel('Step')
plt.ylabel('Reward')
plt.title('Rewards During Episode')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"Average reward per step: {np.mean(rewards):.4f}")
print(f"Max reward: {np.max(rewards):.4f}")
print(f"Min reward: {np.min(rewards):.4f}")

## Explore Action and Observation Spaces

In [ ]:
# Sample and visualize some actions
print("Sample actions from the action space:")
for i in range(5):
    action = env.action_space.sample()
    print(f"Action {i+1}: shape={action.shape}, values={action}")

print(f"\nAction space bounds:")
print(f"Low: {env.action_space.low}")
print(f"High: {env.action_space.high}")
print(f"\nAction space represents control for both arms of the ALOHA robot.")
print(f"Typically includes joint positions/velocities for all 14 joints (7 per arm).")

## Try Different Task (Optional)

You can easily switch between different ALOHA tasks by changing the task name.

In [ ]:
# Close current environment
env.close()

# Try a different task
task = "AlohaTransferCube-v0"  # Switch to cube transfer task
env = gym.make(f'gym_aloha/{task}', render_mode='rgb_array')

observation, info = env.reset(seed=42)
frame = env.render()

plt.figure(figsize=(10, 8))
plt.imshow(frame)
plt.title(f"Initial State of {task}")
plt.axis('off')
plt.show()

print(f"Switched to: {task}")

## Clean Up

In [ ]:
# Close the environment
env.close()
print("Environment closed successfully!")

## Next Steps

Now that you've explored the ALOHA environment, you can:

1. **Train a policy** - Use LeRobot's ACT (Action Chunking Transformer) policy, which was specifically designed for ALOHA
2. **Load demonstrations** - Load pre-recorded expert demonstrations from Hugging Face:
   - `lerobot/aloha_sim_insertion_human`
   - `lerobot/aloha_sim_insertion_scripted`
   - `lerobot/aloha_sim_transfer_cube_human`
   - `lerobot/aloha_sim_transfer_cube_scripted`
3. **Collect your own data** - Record demonstrations for imitation learning
4. **Evaluate policies** - Test trained ACT models on the environment
5. **Bimanual manipulation** - Explore coordinated control of both robot arms

### Example: Loading a dataset

```python
from lerobot.common.datasets.lerobot_dataset import LeRobotDataset

dataset = LeRobotDataset("lerobot/aloha_sim_insertion_human")
print(f"Dataset has {len(dataset)} frames")
```

### Example: Training ACT policy

```bash
python lerobot/scripts/train.py \
    policy=act \
    env=aloha \
    env.task=AlohaInsertion-v0 \
    dataset_repo_id=lerobot/aloha_sim_insertion_human
```

Check out the [LeRobot documentation](https://github.com/huggingface/lerobot) for more examples!